### 🌊 Week 2 Interactive Notebook — Starter

#### 1️⃣ Notebook Outline

- 1. Setup
- 2. Load initial data (simulate Postgres table)
- 3. Read last watermark
- 4. Extract new rows
- 5. Transform
- 6. Load & update watermark
- 7. Metrics demonstration
- 8. Interactive rerun

In [ ]:
import pandas as pd
import json
from pathlib import Path
from typing import Union

In [17]:
WATERMARK_FILE = Path('watermark.json')
OUTPUT_FILE = Path('cdc.csv')

In [4]:
def read_watermark() -> pd.Timestamp:
	if WATERMARK_FILE.exists():
		with open(WATERMARK_FILE, 'r') as f:
			data = json.load(f)
			return pd.Timestamp(data["last_updated"])
	return pd.Timestamp("1970-01-01 00:00:00")

In [ ]:
def write_watermark(timestamp: pd.Timestamp) -> None:
	with open(WATERMARK_FILE, 'w') as f:
		json.dump({"last_updated": timestamp.isoformat()}, f)


In [9]:
def extract_new_rows(df: pd.DataFrame, last_watermark: pd.Timestamp) -> pd.DataFrame:
	"Return only rows updated after the last watermark."
	return df[df['updated_at'] > last_watermark]

In [8]:
def extract(path: Union[str, Path]) -> pd.DataFrame:
	"""Full extract function reading CSV and applying watermark."""
	df = pd.read_csv(path, parse_dates=['updated_at'])
	last_watermark = read_watermark()
	new_rows = extract_new_rows(df, last_watermark)
	return new_rows

In [10]:
def transform(df: pd.DataFrame) -> pd.DataFrame:
	"""Sample transormation: add a processed timestamp"""
	df['total_price'] = df['amount'] * df['unit_price']
	df['processed_at'] = pd.Timestamp.now()
	return df

In [11]:
def load(df: pd.DataFrame, output_path: str = OUTPUT_FILE) -> None:
	if df.empty:
		return
	
	# Append to output
	df.to_csv(output_path, mode='a', index=False, header=not output_path.exists())

	# Update watermark
	max_ts = df['updated_at'].max()
	write_watermark(max_ts)

In [18]:
OUTPUT_FILE

PosixPath('cdc.csv')

In [19]:
read_watermark()

Timestamp('2023-06-01 00:00:00')

In [22]:
def run():
	df = extract('sales.csv')
	print(df.shape[0])
	transformed_df = transform(df)
	load(transformed_df, OUTPUT_FILE)


run()

2


In [45]:
pd.read_parquet('/Users/fg/Library/CloudStorage/OneDrive-Personal/Learn/DataEngineering/senior-de-bootcamp/output')

,id,firstname,amount,unit_price,updated_at,total_price,processed_at,updated_date
0,1,fadeel,10,100,2024-06-01 10:00:00,1000,2026-02-26 17:29:39.497472,2024-06-01
1,2,alice,5,200,2024-06-01 11:00:00,1000,2026-02-26 17:29:39.497472,2024-06-01
2,3,bob,8,150,2024-06-01 12:00:00,1200,2026-02-26 17:29:39.497472,2024-06-01
3,4,charlie,12,120,2024-06-01 13:00:00,1440,2026-02-26 17:29:39.497472,2024-06-01
4,5,dave,7,180,2024-06-01 14:00:00,1260,2026-02-26 17:29:39.497472,2024-06-01
5,6,eve,15,90,2024-06-01 15:00:00,1350,2026-02-26 17:29:39.497472,2024-06-01
6,7,frank,20,80,2024-06-01 16:00:00,1600,2026-02-26 17:29:39.497472,2024-06-01
7,8,grace,9,110,2024-06-01 17:00:00,990,2026-02-26 17:29:39.497472,2024-06-01
8,9,heidi,6,130,2024-06-01 18:00:00,780,2026-02-26 17:29:39.497472,2024-06-01
9,10,ivan,11,140,2024-06-01 18:00:00,1540,2026-02-26 17:29:39.497472,2024-06-01


In [47]:
pd.read_parquet('/Users/fg/Library/CloudStorage/OneDrive-Personal/Learn/DataEngineering/senior-de-bootcamp/output')

,id,firstname,amount,unit_price,updated_at,total_price,processed_at,updated_date
0,1,fadeel,10,100,2024-06-01 10:00:00,1000,2026-02-26 17:29:39.497472,2024-06-01
1,2,alice,5,200,2024-06-01 11:00:00,1000,2026-02-26 17:29:39.497472,2024-06-01
2,3,bob,8,150,2024-06-01 12:00:00,1200,2026-02-26 17:29:39.497472,2024-06-01
3,4,charlie,12,120,2024-06-01 13:00:00,1440,2026-02-26 17:29:39.497472,2024-06-01
4,5,dave,7,180,2024-06-01 14:00:00,1260,2026-02-26 17:29:39.497472,2024-06-01
5,6,eve,15,90,2024-06-01 15:00:00,1350,2026-02-26 17:29:39.497472,2024-06-01
6,7,frank,20,80,2024-06-01 16:00:00,1600,2026-02-26 17:29:39.497472,2024-06-01
7,8,grace,9,110,2024-06-01 17:00:00,990,2026-02-26 17:29:39.497472,2024-06-01
8,9,heidi,6,130,2024-06-01 18:00:00,780,2026-02-26 17:29:39.497472,2024-06-01
9,10,ivan,11,140,2024-06-01 18:00:00,1540,2026-02-26 17:29:39.497472,2024-06-01
